# Inter-patient dataset

This example demostrates how to create inter-patient datasets. It is useful for heartbeat classification task. It is available on [GitHub](https://github.com/devnums/pyheartlib/blob/main/examples/).

<a target="_blank" href="https://colab.research.google.com/github/devnums/pyheartlib/blob/main/examples/dataset/inter_patient.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install pyheartlib

In [1]:
try:
    import pyheartlib
    print('pyheartlib is already installed')
except ModuleNotFoundError:
    print('Installing pyheartlib...')
    !pip install pyheartlib

pyheartlib is already installed


### Download raw data

In [ ]:
# download the raw data and store them in the data directory
from pathlib import Path
if not Path('data').is_dir():
  print('downloading raw data...')
  import io, zipfile
  from urllib.request import urlopen
  url='https://www.physionet.org/static/published-projects/mitdb/mit-bih-arrhythmia-database-1.0.0.zip'
  with urlopen(url) as rs:
      zipf = zipfile.ZipFile(io.BytesIO(rs.read()))
      zipf.extractall('data/')

  # create the config file, for this example just download it from the original repository
  with urlopen("https://raw.githubusercontent.com/devnums/pyheartlib/main/src/pyheartlib/config.yaml") as file:
      content = file.read().decode()
  with open("data/config.yaml", 'w') as file:
      file.write(content)

### Create dataset

In [ ]:
from pyheartlib.data_beat import BeatData
from pyheartlib.beat_info import BeatInfo


# Inter-patient
# Lets create a BeatData object.
beatdata = BeatData(
    base_path="data",
    win=[200, 200],
    remove_bl=False,
    lowpass=False,
    progress_bar=False,
)

# Create a BeatInfo object for features calculations.
beatinfo = BeatInfo()

# Define custom features
def F_new_feature1(self):
    return 1.09


def F_new_feature2(self):
    post = beatinfo.F_post_rri()
    pre = beatinfo.F_pre_rri()
    ret = {"Post": post, "Pre": pre}
    return ret

# Add custom features
new_features = [F_new_feature1, F_new_feature2]
beatinfo.add_features(new_features)

# Get a list of available features
print(beatinfo.available_features())

# Select the desired features
beatinfo.select_features(
    ["F_beat_max", "F_beat_skewness", "F_new_feature1", "F_new_feature2"]
)

['F_absmean_sdrri', 'F_beat_kurtosis', 'F_beat_max', 'F_beat_mean', 'F_beat_min', 'F_beat_rms', 'F_beat_skewness', 'F_beat_std', 'F_compensate_diff_nr', 'F_compensate_ratio', 'F_diff_post_avg_nr', 'F_diff_post_pre', 'F_diff_post_pre_nr', 'F_diff_pre_avg_nr', 'F_fft_features', 'F_heart_rate', 'F_maxmin_diff', 'F_maxmin_diff_norm', 'F_mean_pre_rri', 'F_mean_pre_sdrri', 'F_mean_sdrri', 'F_median_rri', 'F_new_feature1', 'F_new_feature2', 'F_nsampels', 'F_post_rri', 'F_pre_rri', 'F_ratio_post_pre', 'F_ratio_post_rms', 'F_ratio_pre_rms', 'F_rms_rri', 'F_rms_sdrri', 'F_std_rri', 'F_std_sdrri', 'F_subsegs']


In [ ]:
# Use the save_dataset_inter method to create the dataset file.
# The file will be saved in the base data directory.
beatdata.save_dataset_inter(["209"], beatinfo, file="train.beat")

File saved at: data/train.beat


In [ ]:
# In a similar way for validation and test sets
beatdata.save_dataset_inter(["215", "220"], beatinfo, file="val.beat")
beatdata.save_dataset_inter(["103"], beatinfo, file="test.beat")

File saved at: data/val.beat
File saved at: data/test.beat


In [ ]:
# Load processed datasets
train_ds = beatdata.load_data(file_name="train.beat")
val_ds = beatdata.load_data(file_name="val.beat")
test_ds = beatdata.load_data(file_name="test.beat")

File loaded from: data/train.beat
-Shape of "waveforms" is (2985, 400). Number of samples is 2985.
-Shape of "beat_feats" is (2985, 5). Number of samples is 2985.
-Shape of "labels" is (2985,). Number of samples is 2985.
               N  L  R  j  e  V  E    A  S  a  J  F  f  /  Q
train.beat  2601  0  0  0  0  1  0  383  0  0  0  0  0  0  0

File loaded from: data/val.beat
-Shape of "waveforms" is (5371, 400). Number of samples is 5371.
-Shape of "beat_feats" is (5371, 5). Number of samples is 5371.
-Shape of "labels" is (5371,). Number of samples is 5371.
             N  L  R  j  e    V  E   A  S  a  J  F  f  /  Q
val.beat  5109  0  0  0  0  164  0  97  0  0  0  1  0  0  0

File loaded from: data/test.beat
-Shape of "waveforms" is (2064, 400). Number of samples is 2064.
-Shape of "beat_feats" is (2064, 5). Number of samples is 2064.
-Shape of "labels" is (2064,). Number of samples is 2064.
              N  L  R  j  e  V  E  A  S  a  J  F  f  /  Q
test.beat  2062  0  0  0  0  0  0  2  

In [ ]:
# Number of samples per class
stat_report = beatdata.report_stats_table(
    [train_ds["labels"], val_ds["labels"], test_ds["labels"]], ["Train", "Val", "Test"]
)
print(stat_report)

          N  L  R  j  e    V  E    A  S  a  J  F  f  /  Q
Train  2601  0  0  0  0    1  0  383  0  0  0  0  0  0  0
Val    5109  0  0  0  0  164  0   97  0  0  0  1  0  0  0
Test   2062  0  0  0  0    0  0    2  0  0  0  0  0  0  0


In [ ]:
print(train_ds["beat_feats"].tail(3))

      F_beat_max  F_beat_skewness  F_new_feature1        Post         Pre
2982       1.250         2.189705            1.09  641.666667  663.888889
2983       1.355         1.755593            1.09  494.444444  641.666667
2984       1.420         2.580627            1.09  761.111111  494.444444
